In [92]:
from functools import total_ordering
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as soup
import pandas as pd
import requests
import time
import os
import numpy as np


In [93]:
user_zip = input('Enter a zip code and press enter ')
print(f'Searching for Utilities serving  {user_zip} and the surrrounding cities for your county')

Searching for Utilities serving  59714 and the surrrounding cities for your county


In [94]:
page="https://www.ewg.org/tapwater/search-results.php?zip5="+ str(user_zip)+"&searchtype=zip"
url = requests.get(page)
table = pd.read_html(url.text)

In [95]:
table[0]

,Utility name,City,People served
0,City of Bozeman,"Bozeman, MT","Population served:32,000"
1,"West Yellowstone, Town of","West Yellowstone, MT","Population served:8,535"
2,"Belgrade, City of","Belgrade, MT","Population served:7,000"
3,River Rock County Water and Sewer District,"Belgrade, MT","Population served:4,200"
4,"Three Forks, City of","Three Forks, MT","Population served:1,900"
5,North Star Zoot Water,"Bozeman, MT","Population served:3,000"
6,"Manhattan, Town of","Manhattan, MT","Population served:1,800"
7,Rae Water and Sewer District 313,"Bozeman, MT","Population served:1,130"
8,Valley Grove Subdivision,"Bozeman, MT",Population served:825
9,Landmark Subdivision,"Bozeman, MT",Population served:750


In [96]:
table[1]

,Utility name,City,People served
0,Antler Ridge Homeowners Association,Big Sky,Population served: 50
1,Baxter Creek No. 2 Homeowners Association,Bozeman,Population served: 450
2,Belgrade Trailer Court,Bozeman,Population served: 50
3,Belgrade Village Mobile Home Park,Belgrade,Population served: 100
4,"Belgrade, City of",Belgrade,"Population served: 7,000"
5,Bmp Lincoln Rentals LLC,Bozeman,Population served: 31
6,Bridger Base Water System,Belgrade,Population served: 75
7,Churchill North Subdivision,Manhattan,Population served: 155
8,Churchill Retirement Home,Manhattan,Population served: 80
9,City of Bozeman,Bozeman,"Population served: 32,000"


In [69]:
utilities_df = pd.concat([table[0], table[1]], ignore_index=True)

In [70]:
utilities_df

,Utility name,City,People served
0,City of Bozeman,"Bozeman, MT","Population served:32,000"
1,"West Yellowstone, Town of","West Yellowstone, MT","Population served:8,535"
2,"Belgrade, City of","Belgrade, MT","Population served:7,000"
3,River Rock County Water and Sewer District,"Belgrade, MT","Population served:4,200"
4,"Three Forks, City of","Three Forks, MT","Population served:1,900"
...,...,...,...
57,"Three Forks, City of",Three Forks,"Population served: 1,900"
58,Valley Grove Subdivision,Bozeman,Population served: 825
59,"West Yellowstone, Town of",West Yellowstone,"Population served: 8,535"
60,Wylie Creek Estates Subdivision,Bozeman,Population served: 400


In [71]:
#Build a list of the utilities to visit and scrape data from EWG
utility_list = utilities_df['Utility name'].to_list()
utility_list

['City of Bozeman',
 'West Yellowstone, Town of',
 'Belgrade, City of',
 'River Rock County Water and Sewer District',
 'Three Forks, City of',
 'North Star Zoot Water',
 'Manhattan, Town of',
 'Rae Water and Sewer District 313',
 'Valley Grove Subdivision',
 'Landmark Subdivision',
 'Antler Ridge Homeowners Association',
 'Baxter Creek No. 2 Homeowners Association',
 'Belgrade Trailer Court',
 'Belgrade Village Mobile Home Park',
 'Belgrade, City of',
 'Bmp Lincoln Rentals LLC',
 'Bridger Base Water System',
 'Churchill North Subdivision',
 'Churchill Retirement Home',
 'City of Bozeman',
 'Cobblestone Subdivision',
 'Country Court Gallatin',
 'Covered Wagon Mb Hm Park',
 'Dyksterhouse Well Users Association',
 'Elk Grove Water',
 'Firelight Meadows LLC',
 'Forest Creek Subdivision',
 'Forest Park Trailer Court No. 1 Bozeman',
 'Franklin Hills Subdivision',
 'Godfrey Canyon Estates',
 'Gravel Hollow LLC',
 'Hardin Subdivision',
 'Hebgen Lake Estates County W and S District',
 'Hidden 

In [72]:
start_time = time.time()

#Begin Scrape
driver = webdriver.Chrome()

contaminant_list = []
for utility in utility_list:
    try:
        driver.get("https://www.ewg.org/tapwater/advanced-search.php")
        time.sleep(1)
        utility_input = driver.find_element(By.XPATH, '/html/body/div[3]/main/div/section[1]/form/input[1]')
        utility_input.clear()
        utility_input.send_keys(utility)
        go_btn = driver.find_element(By.XPATH, '/html/body/div[3]/main/div/section[1]/form/input[3]')
        go_btn.click()
        time.sleep(1)
        utility = driver.find_element(By.XPATH, '/html/body/div[3]/main/figure/table/tbody/tr/td[1]/a')
        utility.click()

        page_source = driver.page_source

        html_soup = soup(page_source, 'html.parser')

        #Get the name of the Water Utility
        Utility = html_soup.find('h1').text
        # Utility


        #get the html data we need
        data_box = html_soup.find_all('div', class_='contaminant-name')

        for i in range(len(data_box)):
            data = data_box[i].find_all('span')
            data_measure = []
            d = {
                'Utility' : Utility,
                'Contaminant': '', 
                'Utility Measuremnt':'', 
                'EWG HEALTH GUIDELINE': '',
                'Legal Limit':'' 
            }

            contaminant_name = data_box[i].find('h3')
            d['Contaminant'] = contaminant_name

            for j in range(len(data)):
                measurement = data[j].text
                #print(measurement)
                data_measure.append(measurement)
                #print(data_measure)

            try:
                d['Utility Measuremnt'] = data_measure[data_measure.index('THIS UTILITY')+1]
            except ValueError:
                print("A value error arose")
            except:
                print("Something else went wrong")
            try:
                d['EWG HEALTH GUIDELINE'] = data_measure[data_measure.index('EWG HEALTH GUIDELINE')+1]
            except ValueError:
                print("A value error arose")
            except:
                print("Something else went wrong") 
            try:
                d['Legal Limit'] = data_measure[data_measure.index('LEGAL LIMIT')+1]
            except ValueError:
                print("A value error arose")
            except:
                print("Something else went wrong") 

            contaminant_list.append(d)
    except:
        pass
print ('Done Scraping, moving on to dataset cleaning and file writing')
#Construct a dataframe from the results
scraped_df = pd.DataFrame(contaminant_list)

#Define a function to strip the h3 tages from the contaminants (relict from scraping)
import re
CLEANR = re.compile('<.*?>') 

def cleanhtml(raw_html):
    cleantext = re.sub(CLEANR, '', raw_html)
    return cleantext

scraped_df = scraped_df.astype({"Contaminant": str})

scraped_df.Contaminant = scraped_df.Contaminant.apply(lambda x: cleanhtml(x))

# #Output the resulting dataframe to a csv file
# state_path_contaminant_output = os.path.join(path,'contaminants.csv')
# scraped_df.to_csv(state_path_contaminant_output, index=False)

finish_time = time.time()

total_time = (finish_time - start_time)/60

print(f'The process finished in finished in {total_time} minutes')

A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose


In [73]:
scraped_df

,Utility,Contaminant,Utility Measuremnt,EWG HEALTH GUIDELINE,Legal Limit
0,City of Bozeman,Bromochloroacetic acid,0.207 ppb,0.02 ppb,
1,City of Bozeman,Bromodichloromethane,1.18 ppb,0.06 ppb,
2,City of Bozeman,Chloroform,18.1 ppb,0.4 ppb,
3,City of Bozeman,Chromium (hexavalent),0.116 ppb,0.02 ppb,
4,City of Bozeman,Dichloroacetic acid,6.99 ppb,0.2 ppb,
...,...,...,...,...,...
547,Yellowstone Holiday Water and Sew District,Uranium,1.47 pCi/L,0.43 pCi/L,20 pCi/L
548,Yellowstone Holiday Water and Sew District,Barium,79.0 ppb,700 ppb,"2,000 ppb"
549,Yellowstone Holiday Water and Sew District,Fluoride,0.400 ppm,,4 ppm
550,Yellowstone Holiday Water and Sew District,Nitrate and nitrite,0.146 ppm,0.14 ppm,10 ppm


In [74]:
df = scraped_df.copy()
df.head()

,Utility,Contaminant,Utility Measuremnt,EWG HEALTH GUIDELINE,Legal Limit
0,City of Bozeman,Bromochloroacetic acid,0.207 ppb,0.02 ppb,
1,City of Bozeman,Bromodichloromethane,1.18 ppb,0.06 ppb,
2,City of Bozeman,Chloroform,18.1 ppb,0.4 ppb,
3,City of Bozeman,Chromium (hexavalent),0.116 ppb,0.02 ppb,
4,City of Bozeman,Dichloroacetic acid,6.99 ppb,0.2 ppb,


In [75]:
#Drop rows with non EWG Guideline for a contaminant
#df3[df3['Contaminant'].notna()]
#df3['EWG HEALTH GUIDELINE'].notnull().value_counts()
df =df.replace(r'^\s*$', np.nan, regex=True)

In [76]:
df.sample(30)

,Utility,Contaminant,Utility Measuremnt,EWG HEALTH GUIDELINE,Legal Limit
220,Covered Wagon Mb Hm Park,Bromodichloromethane,0.630 ppb,0.06 ppb,NaN
437,River Rock County Water and Sewer District,"Radium, combined (-226 &amp; -228)",0.05 pCi/L,0.05 pCi/L,5 pCi/L
171,Bridger Base Water System,Dichloroacetic acid,5.05 ppb,0.2 ppb,NaN
217,Country Court Gallatin,Nitrate and nitrite,0.188 ppm,0.14 ppm,10 ppm
378,Mt Ellis Academy,Fluoride,0.250 ppm,NaN,4 ppm
146,Belgrade Trailer Court,None,NaN,NaN,NaN
59,"Three Forks, City of",Fluoride,1.57 ppm,NaN,4 ppm
333,Landmark Subdivision,Haloacetic acids (HAA5)†,0.370 ppb,0.1 ppb,60 ppb
203,City of Bozeman,Manganese,4.43 ppb,100 ppb,NaN
83,North Star Zoot Water,None,NaN,NaN,NaN


In [77]:
df = df.dropna()

In [78]:
df.sample(20)

,Utility,Contaminant,Utility Measuremnt,EWG HEALTH GUIDELINE,Legal Limit
519,"Three Forks, City of",Selenium,0.600 ppb,30 ppb,50 ppb
434,River Rock County Water and Sewer District,Barium,100.0 ppb,700 ppb,"2,000 ppb"
287,Franklin Hills Subdivision,Nitrate and nitrite,1.37 ppm,0.14 ppm,10 ppm
239,Dyksterhouse Well Users Association,Nitrate and nitrite,4.68 ppm,0.14 ppm,10 ppm
160,Bmp Lincoln Rentals LLC,Arsenic,1.50 ppb,0.004 ppb,10 ppb
310,Hebgen Lake Estates County W and S District,Barium,8.13 ppb,700 ppb,"2,000 ppb"
300,Gravel Hollow LLC,Nitrate and nitrite,0.442 ppm,0.14 ppm,10 ppm
5,City of Bozeman,Haloacetic acids (HAA5)†,14.2 ppb,0.1 ppb,60 ppb
147,Belgrade Village Mobile Home Park,Arsenic*,0.429 ppb,0.004 ppb,10 ppb
498,Thomson Mobile Home Park,"Radium, combined (-226 &amp; -228)",0.10 pCi/L,0.05 pCi/L,5 pCi/L


In [79]:
df3 = df.copy()
#Split of the contaminant measurements and units and people served
df3['Units'] = df3['Utility Measuremnt'].apply(lambda x: x.split()[-1])
df3['Utility Measuremnt'] = df3['Utility Measuremnt'].apply(lambda x: x.split()[0])
df3['EWG HEALTH GUIDELINE'] = df3['EWG HEALTH GUIDELINE'].apply(lambda x: x.split()[0])
df3['Legal Limit']=df3['Legal Limit'].apply(lambda x: '0 units' if pd.isnull(x) else x)
df3['Legal Limit'] = df3['Legal Limit'].apply(lambda x: x.split()[0])
#df3['People served'] = df3['People served'].apply(lambda x: x.split(':')[-1])

In [80]:
#Replace any commas in the thousandths place
df3.replace(',','', regex=True, inplace=True)

In [81]:
#Change the datatype of the measurements and EWG Guidlines to numeric values
df3['Utility Measuremnt'] = pd.to_numeric(df3['Utility Measuremnt'])
df3['EWG HEALTH GUIDELINE'] = pd.to_numeric(df3['EWG HEALTH GUIDELINE'])
#df3['People served'] = pd.to_numeric(df3['People served'])

In [82]:
#Define the Contaminant Factor (how many times larger is the utility measurement than EWG guideline)
df3['Contaminant_Factor'] = df3['Utility Measuremnt']/df3['EWG HEALTH GUIDELINE']

In [83]:
df3

,Utility,Contaminant,Utility Measuremnt,EWG HEALTH GUIDELINE,Legal Limit,Units,Contaminant_Factor
5,City of Bozeman,Haloacetic acids (HAA5)†,14.200,0.100,60,ppb,142.000000
7,City of Bozeman,Total trihalomethanes (TTHMs)†,19.300,0.150,80,ppb,128.666667
14,City of Bozeman,Nitrate and nitrite,0.080,0.140,10,ppm,0.571429
16,City of Bozeman,Uranium,0.370,0.430,20,pCi/L,0.860465
19,West Yellowstone Town of,Arsenic,1.310,0.004,10,ppb,327.500000
...,...,...,...,...,...,...,...
543,Wylie Creek Estates Subdivision,Barium,85.000,700.000,2000,ppb,0.121429
546,Yellowstone Holiday Water and Sew District,Radium combined (-226 &amp; -228),1.250,0.050,5,pCi/L,25.000000
547,Yellowstone Holiday Water and Sew District,Uranium,1.470,0.430,20,pCi/L,3.418605
548,Yellowstone Holiday Water and Sew District,Barium,79.000,700.000,2000,ppb,0.112857


In [84]:
utilities_grouped_df1 = df3.groupby('Utility').count()['Contaminant_Factor']
utilities_grouped_df2 = df3.groupby('Utility').sum()['Contaminant_Factor']


In [85]:
utilities_grouped_df1

Utility
All Star Mobile Home Park                         2
Antler Ridge Homeowners Association               4
Baxter Creek No. 2 Homeowners Association         4
Belgrade City of                                 10
Belgrade Trailer Court                            4
Belgrade Village Mobile Home Park                 5
Black Mountain Meadow Condos                      2
Bmp Lincoln Rentals LLC                           4
Bridger Base Water System                         5
Brookfield-no Riverside Water Commission          6
Churchill North Subdivision                       6
Churchill Retirement Home                         3
City of Bozeman                                   8
Cobblestone Subdivision                           2
Country Court Gallatin                            1
Covered Wagon Mb Hm Park                          4
Dyksterhouse Well Users Association               6
Elk Grove Water                                   6
Firelight Meadows LLC                             8
Fore

In [86]:
utilities_grouped_df2

Utility
All Star Mobile Home Park                          31.581395
Antler Ridge Homeowners Association               710.052326
Baxter Creek No. 2 Homeowners Association         523.618272
Belgrade City of                                  260.177409
Belgrade Trailer Court                             32.188704
Belgrade Village Mobile Home Park                 130.088704
Black Mountain Meadow Condos                     1150.067083
Bmp Lincoln Rentals LLC                           401.518272
Bridger Base Water System                         171.112708
Brookfield-no Riverside Water Commission          460.705333
Churchill North Subdivision                      1269.509468
Churchill Retirement Home                          60.192691
City of Bozeman                                   544.197121
Cobblestone Subdivision                            19.600000
Country Court Gallatin                              1.342857
Covered Wagon Mb Hm Park                          102.265227
Dyksterhouse Wel

In [87]:
Sum_ContaminantFactor = utilities_grouped_df2.sum()
Avg_Contaminant_Factor = utilities_grouped_df2.mean()
Num_Contaminants = utilities_grouped_df1.sum()

In [88]:
#Output to be joined with census data for racial, ethnic, and income indices
data = {
    'Zip' : user_zip,
    'Sum_ContaminantFactor': Sum_ContaminantFactor, 
    'Avg_Contaminant_Factor':Avg_Contaminant_Factor, 
    'Num_Contaminants' : Num_Contaminants
}

df_final = pd.DataFrame(data, index=[0])

In [89]:
df_final

,Zip,Sum_ContaminantFactor,Avg_Contaminant_Factor,Num_Contaminants
0,59714,21739.383617,426.262424,278


In [97]:
type(df_final)

pandas.core.frame.DataFrame

In [104]:
df_final.loc[0,'Num_Contaminants']

278

In [105]:
df_final.dtypes

Zip                        object
Sum_ContaminantFactor     float64
Avg_Contaminant_Factor    float64
Num_Contaminants            int64
dtype: object